## This sample choose you own adventure (CYOA) application demonstrates the following
1) How to use the LangChain python library to interact programatically with OpenAI Chatbot
2) How to implement Retrieval Augmented Generation (RAG) using in-memory vectorstore called FAISS
3) How to use private context taken from the web - loading gear from dndbeyond.com

In [24]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import chat_messages

In [25]:
load_dotenv()
llm = ChatOpenAI()

In [26]:
# initialize the CYOA prompt with system message to set the context
prompt = ChatPromptTemplate.from_messages([
    ("system", chat_messages.system_message.format(chat_history=chat_history)),
    ("user", "{choice}")
])

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"choice":""})

'Hello, brave traveler! Welcome to the enchanted realm. Before we embark on this thrilling journey, I must ask you to select a legendary weapon from the armory. Which of these options calls to your heart?\n\n1) The Blade of Aurora - a gleaming sword said to possess the power of the sun.\n2) The Whispering Bow - a bow that can shoot arrows that never miss their target.\n3) The Staff of Zephyr - a staff that controls the winds and storms.\n\nChoose wisely, for your weapon shall aid you in the challenges ahead.'

## Implement the retrieval chain
1) Scrape armory from dndbeyond.com using WebBaseLoader and beautifulsoup4
2) Load armory into faiss vector store 

In [33]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader('https://www.dndbeyond.com/equipment?filter-search=Weapon')
armory = loader.load()

print(armory)

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(armory)

vectorstore = FAISS.from_documents(documents, embeddings)

[Document(page_content='\n\n\n\n\n\n\n\n\n\n\nEquipment, Gear, and Items for Dungeons & Dragons (D&D) Fifth Edition (5e) - D&D Beyond \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n This site works best with JavaScript enabled. Please enable JavaScript to get the best experience from this site.\n\n\nSkip to Content\n\n\n\n\n\n\nD&D Beyond\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSign In\nRegister\n\n\n\n\n\n\n\n\n\n\n\nD&D Beyond\n\n\n\n\n\n\n\n\n\r\n                   Collections\r\n    \r\n                \n\n\n\n\n\r\n                   My Characters\r\n    \r\n                \n\n\n\n\n\r\n                   My Campaigns\r\n    \r\n                \n\n\n\n\n\r\n                   My Encounters\r\n    \r\n                \n\n\n\n\n\r\n                   My Dice\r\n    \r\n                \n\n\n\n\n\r\n                   My Homebrew Collection\r\n    \r\n                \n\n\n\n\n\r\n                   My Homebrew Creations

## Load armory into embeddings

In [49]:
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""

prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)

from langchain_core.documents import Document

document_chain.invoke({
    "input":"List the names of the first 3 weapons",
    "context": [Document(page_content="List of weapons: 1) Axe 2) Slingshot, 3) Halbred 4) Sword 5) Javelin")]
})


'The names of the first 3 weapons are Axe, Slingshot, and Halbred.'

In [50]:
from langchain.chains import create_retrieval_chain

retriever = vectorstore.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({
    "input" : "List the names of all the available weapons."
})

response

{'input': 'Enumerate the names of all the available weapons.',
 'context': [Document(page_content='View Magic Items\n        \n\n\n\n\n\n\n\n\n\n              Equipment\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n                Search\r\n            \n\n\n\n\n\n\n\n\r\n                Cost\r\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n                Weight\r\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\nFilter Equipment\n\n\n\n\n\n\nShow Advanced Filters\n\n\n\n\nReset All Filters\n\n\n\n\nName\nCost\nWeight\nAttributes\nNotes\n\n\n\n\n\n12Next\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                            Antimatter Rifle\n                        \n\n\n\n\n                        Firearms Ranged Weapon\n                    \n\n\n\n\n\n\n\n                         -- \n                    \n\n\n\n\n\n\n\n                        10 lbs\n                    \n\n\n\n\n\n\n\n                        Necrotic\n                    \n\n\n\n\n\n\n\n               

### Conversational retrieval chain

In [88]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get a list of weapons")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Can you find different types of weapons"),
    AIMessage(content="Yes.")
]

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the users questions based on the below contenxt:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
   ("user", "input")
])

document_chain = create_stuff_documents_chain(llm, prompt)

conversation_retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

response = conversation_retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "List just the names of all the weapons you find in following the format: 1. Weapon 1\n2. Weapon 2"
})

response


In [87]:
list = response.get('answer').split('\n')

[l for l in list if 'Weapon' in l]

['1. Hand Mortar (Exandria) - Martial Melee Weapon',
 '2. Handaxe - Simple Melee Weapon',
 '3. Hoopak - Martial Melee Weapon',
 '4. Hoopak (Ranged) - Martial Ranged Weapon',
 '5. Javelin - Simple Melee Weapon',
 '6. Lance - Martial Melee Weapon',
 '7. Laser Pistol - Firearms Ranged Weapon',
 '8. Laser Rifle - Firearms Ranged Weapon',
 '9. Light Hammer - Simple Melee Weapon',
 '10. Longbow - Martial Ranged Weapon',
 '11. Crossbow, light - Simple Ranged Weapon',
 '12. Dagger - Simple Melee Weapon',
 '13. Dart - Simple Ranged Weapon',
 '14. Double-Bladed Scimitar - Martial Melee Weapon',
 '15. Flail - Martial Melee Weapon',
 '16. Glaive - Martial Melee Weapon',
 '17. Greataxe - Martial Melee Weapon',
 '18. Greatclub - Simple Melee Weapon',
 '19. Greatsword - Martial Melee Weapon',
 '20. Halberd - Martial Melee Weapon',
 '21. Longsword - Martial Melee Weapon',
 '22. Battleaxe - Martial Melee Weapon',
 '23. Blowgun - Martial Ranged Weapon',
 '24. Club - Simple Melee Weapon',
 '25. Crossbow,